In [1]:
pip install tensorflow mlflow scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 26.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.1/554.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.7/626.7 kB 8.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import mlflow
import mlflow.tensorflow
from mlflow.models import ModelSignature
from mlflow.models.signature import infer_signature

2024-09-17 17:08:45.277148: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 17:08:45.281410: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 17:08:45.290339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 17:08:45.304154: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 17:08:45.309428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 17:08:45.337696: I tensorflow/core/platform/cpu_feature_gu

In [ ]:
import warnings
import os

# Suppress warnings and TensorFlow logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')

import tensorflow as tf
import tensorflow_datasets as tfds
import mlflow
import mlflow.tensorflow
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
import numpy as np
from mlflow.models.signature import infer_signature

# 1. Set the MLflow experiment name (it will create one if it doesn't exist)
mlflow.set_experiment("Flower_Classification_Experiment")

# 2. Load and preprocess the data
dataset, info = tfds.load('tf_flowers', with_info=True, as_supervised=True)

train_data = dataset['train']
test_data = dataset['train'].take(1000)  # Use 1000 samples for testing
train_data = dataset['train'].skip(1000) # Rest for training

label_names = info.features['label'].names

def preprocess_image(image, label):
    image = tf.image.resize(image, [128, 128])
    image = image / 255.0
    return image, label

train_data = train_data.map(preprocess_image).shuffle(1000).batch(32).prefetch(1)
test_data = test_data.map(preprocess_image).batch(32).prefetch(1)

# 3. Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 4. Start an MLflow experiment and log model details
with mlflow.start_run() as run:
    mlflow.tensorflow.autolog()  # Automatically log metrics and parameters
    
    # Train the model
    history = model.fit(train_data, epochs=5, validation_data=test_data)
    
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test_data)
    
    # 5. Make predictions and calculate additional metrics (precision, recall, f1 score)
    y_true = []
    y_pred = []

    for images, labels in test_data:
        predictions = model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(predictions, axis=1))
    
    # Calculate precision, recall, and F1-score using sklearn
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)

    # Log metrics manually to MLflow
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)

    # Obtain a single batch of data for model signature inference
    for images, _ in test_data.take(1):  # Take only one batch
        example_input = images.numpy()
        break

    # Create a model signature
    example_output = model.predict(example_input)
    signature = infer_signature(example_input, example_output)
    mlflow.tensorflow.log_model(model, "flower_classifier_model", signature=signature)
    
    print(f"Model saved in run {run.info.run_id}")

# 6. Get MLflow run information and register the model in the model registry
mlflow_model_uri = f"runs:/{run.info.run_id}/flower_classifier_model"
mlflow.register_model(model_uri=mlflow_model_uri, name="Flower_Classifier_Model")

2024-09-17 17:44:30.161023: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 516ms/step - accuracy: 0.3038 - loss: 1.4988

84/84 ━━━━━━━━━━━━━━━━━━━━ 50s 577ms/step - accuracy: 0.3050 - loss: 1.4968 - val_accuracy: 0.5420 - val_loss: 1.1089
Epoch 2/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - accuracy: 0.5700 - loss: 1.0786

84/84 ━━━━━━━━━━━━━━━━━━━━ 48s 571ms/step - accuracy: 0.5704 - loss: 1.0777 - val_accuracy: 0.5980 - val_loss: 1.0118
Epoch 3/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - accuracy: 0.6514 - loss: 0.8866

84/84 ━━━━━━━━━━━━━━━━━━━━ 49s 578ms/step - accuracy: 0.6516 - loss: 0.8861 - val_accuracy: 0.6140 - val_loss: 0.9485
Epoch 4/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 86s 632ms/step - accuracy: 0.7290 - loss: 0.7138 - val_accuracy: 0.6520 - val_loss: 0.9665
Epoch 5/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 78s 584ms/step - accuracy: 0.8115 - loss: 0.5319 - val_accuracy: 0.6250 - val_loss: 1.1311
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
15/32 ━━━━━━━━━━━━━━━━━━━━ 2s 161ms/step - accuracy: 0.6479 - loss: 1.0473